In [1]:
class fdict(dict):
    """Pseudo-functional datastructure, combining a dict and an linked list. The elements are only added into the top dictionary, making no modification to others ones. Use the method infant to create en child from a fdict. And use the todict method to flatten the fdict.
    
    
    This is obviously not optimized, cache efficient, or any other smart trics, but should do the job for easy applications. It should be garbaged collected normally when freed (it does not create cycles) as long as you don't store it inside itself!

    It is only pseudo-functionnal, if you modify an lower layer, it will modify the view of each outer layers.
    
    Example usage:
    a = fdict()
    
    > a[1] = 1
    > a[2] = 2
    > 
    > b = a.infant()
    > b[1] = 3
    > assert b[1] == 3
    > assert b[2] == 2
    > assert a[1] == 1
    > 
    > try:
    >     b[3] # raises KeyError
    > except KeyError:
    >     pass #ok
    > 
    > print(a)
    > print(b)
    > print(b.todict())
    {1: 1, 2: 2}
    {1: 3}
    {1: 3, 2: 2}
    """
    def __init__(self, *args, **kwargs):
        self.parent = None
        return super().__init__(*args, **kwargs)

    def __getitem__(self, key):
        if self.parent:
            try:
                return super().__getitem__(key)
            except KeyError:
                return self.parent.__getitem__(key)
        else:
            return super().__getitem__(key)
    
    def infant(self):
        result = fdict()
        result.parent = self
        return result
    
    def todict(self) -> dict:
        if self.parent:
            d = self.parent.todict()
            d.update(self)
            return d
        else:
            return dict(self)


In [2]:
import heapq

""" precise belt placement """

def heuristic(a, b):
    """Calculate the p distance (with p = 1/2) between a and b.

    Using p = 1/2 permits to favorize solutions that try to be as octolinear as possible."""
    return (abs(a[0] - b[0])**0.5 + abs(a[1] - b[1])**0.5)**2

# type state = { mask: fdict int * int -> bool; position: int*int }

def a_star(start, goal, mask):
    """A* algorithm implementation."""
    # The open list is a priority queue (heap)
    open_list = []
    heapq.heappush(open_list, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}
    info = {start: None}

    while open_list:
        current_cost, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            infos = []
            while current is not None:
                path.append(current)
                infos.append(info[current])
                current = came_from[current]

            
            infos.pop()
            
            return list(zip(path[::-1], infos[::-1]))

        for next, cost, data in neighbours(current, mask):
            new_cost = cost_so_far[current] + cost  # Assuming uniform cost
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                info[next] = data
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(goal, next)
                heapq.heappush(open_list, (priority, next))
                came_from[next] = current

    return None  # Path not found


def neighbours(node, mask):
    """Generate neighbors for a node considering the mask."""
    directions = [
        ("belt", (1, 0)),
        ("belt", (-1, 0)),
        ("belt", (0, 1)),
        ("belt", (0, -1)),
        ("underground", (4, 0)),
        ("underground", (5, 0)),
        ("underground", (6, 0)),
        ("underground", (-4, 0)),
        ("underground", (-5, 0)),
        ("underground", (-6, 0)),
        ("underground", (0, 4)),
        ("underground", (0, 5)),
        ("underground", (0, 6)),
        ("underground", (0, -4)),
        ("underground", (0, -5)),
        ("underground", (0, -6)),
    ]

    kind_cost = {"belt": 1, "underground": 10}

    for kind, (dx, dy) in directions:
        next_node = (node[0] + dx, node[1] + dy)
        ok = mask[next_node] == False
        for ddx in range(x, x + dx):
            for ddy in range(y, y + dy):
                pass


        if next_node in mask and not mask[next_node] and ok:
            yield (next_node, kind_cost[kind], (kind, (dx, dy)))



IndentationError: expected an indented block after 'for' statement on line 76 (854121056.py, line 79)

In [3]:
from itertools import product

# True indicates an impassable area
mask = {(x, y): False for x, y in product(range(11), repeat=2)}

for pos, data in a_star((0, 5), (10, 5), mask):
    assert not (mask[pos])
    mask[pos] = data

for pos, data in a_star((5, 0), (5, 10), mask):
    assert not (mask[pos])
    mask[pos] = data

for pos, data in a_star((6, 0), (4, 10), mask):
    assert not (mask[pos])
    mask[pos] = data


NameError: name 'a_star' is not defined

In [37]:
from PIL import Image, ImageDraw, ImageDraw2
import numpy as np

im = Image.fromarray(255 - np.zeros((110, 110)))
draw = ImageDraw.Draw(im)

for (x, y), data in mask.items():
    if not data:
        continue
    
    kind, (dx, dy) = data

    draw.line((10*x, 10*y, 10*(x+dx), 10*(y+dy)), fill=128)

im.show()